In [1]:
# tag 프로젝트에서 수집된 데이터중 쓸만한 부분을 떼어네서 테스트중

# gensim library

In [1]:
import gensim
print(gensim.__version__)

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


3.4.0


In [2]:
import soynlp

In [3]:
data_fpath = '../data/ta_hashtag/hashtag_test.txt' # tag text
TRAIN_WORD2VEC = False # 학습 switch 
word2vec_fpath = '../data/ta_hashtag/hashtag_word2vec_model_test.pkl' # 학습데이터

# preprocessing
#### text 토크나이징, 띄어쓰기수정, 맞춤법수정등 
#### tag데이터는 일상어에 비해 단어로 이루어진경우가 많으므로 그대로 진행함

In [4]:
def get_text(fname, debug=True):
    with open(fname, encoding='utf-8', errors='ignore') as f:
        docs = []
        for i, doc in enumerate(f):
            if debug and i >= 10:
                break                
#             doc = doc.replace(',',' ')
            doc = doc.replace('\n','')                                
            docs.append(doc)
    return docs

docs = get_text(data_fpath)
print(docs[:20])

['팔 명지대학교 그린나래 기획봉사 농촌활동 내', '아역모델 작시 zaxy 작시슈즈 엠드림즈 브랜드 ootd 패피 신발 광고모델 김하은', '보이차 아지트샵 아지트샵조히', '무한체력 엄마는 너가 조용하면 좀 무섭다', '감사해 에보나이트 그래도빈손아니니까', '시바스타그램 kyoto shibainu kaji uruwasiki shibakaji shiba 시바견 시바 시바이누 시바개 견스타그램 독스타그램 도그스타그램', '스타패션 imjinah nana 임진아 나나 킬잇 드라마패션 아보아보 avouavou kstyle killit imjinahstyle nanastyle kdramastyle kdrama_fashion kfashion 연예인패션 연예인착용 연예인스타일 임진아패션 임진아스타일 임진아원피스 나나패션 나나원피스 패션정보 드라마패션그램', '원본 셀카', '양세형', '울산대']


In [5]:
# word2vec 학습
# class로 iterator를 오버라이딩해 for문을 통해 한줄씩 실행하는 구조가 낯설다..

In [6]:
import os

class TagWord2Vec:
    def __init__(self, fname):
        self.fname = fname
        if not os.path.exists(fname):
            print('File not found: %s' % fname)        
                
    def __iter__(self):        
        with open(self.fname, encoding='utf-8', errors='ignore') as f:                        
            
            for doc in f:            
                doc = doc.replace('\n','')
                yield doc.split(' ')        

            
word2vec_corpus = TagWord2Vec(data_fpath)


In [7]:
for i, doc in enumerate(word2vec_corpus):
    if i > 5 : break
    print(doc)

['팔', '명지대학교', '그린나래', '기획봉사', '농촌활동', '내']
['아역모델', '작시', 'zaxy', '작시슈즈', '엠드림즈', '브랜드', 'ootd', '패피', '신발', '광고모델', '김하은']
['보이차', '아지트샵', '아지트샵조히']
['무한체력', '엄마는', '너가', '조용하면', '좀', '무섭다']
['감사해', '에보나이트', '그래도빈손아니니까']
['시바스타그램', 'kyoto', 'shibainu', 'kaji', 'uruwasiki', 'shibakaji', 'shiba', '시바견', '시바', '시바이누', '시바개', '견스타그램', '독스타그램', '도그스타그램']


In [8]:
# 학습된 데이터를 

In [9]:
%%time
from gensim.models import Word2Vec
import pickle

if TRAIN_WORD2VEC:
    word2vec_model = Word2Vec(
        word2vec_corpus,
        size=100,
        alpha=0.025,
        window=5,
        min_count=5,
        workers=3,
        sg=0,
        negative=5
    )
    with open(word2vec_fpath,'wb') as f: #미리 학습된 내용..
        pickle.dump(word2vec_model, f)
        
else:
    with open(word2vec_fpath, 'rb') as f:
        word2vec_model = pickle.load(f)
        
                

Wall time: 434 ms


In [10]:
word2vec_model.wv.most_similar('존맛', topn=30)

[('존맛탱', 0.9569051265716553),
 ('꿀맛', 0.9443696737289429),
 ('먹방샷', 0.9419482350349426),
 ('맛있어', 0.9391810894012451),
 ('배불러', 0.9370462894439697),
 ('고기그램', 0.930396318435669),
 ('jmtgr', 0.9298840761184692),
 ('존맛탱구리', 0.9225170612335205),
 ('맛있다', 0.9218858480453491),
 ('또먹고싶다', 0.9205896854400635),
 ('엽기떡볶이', 0.9176554679870605),
 ('파스타', 0.9165036678314209),
 ('냠냠', 0.9160664677619934),
 ('jmt', 0.9155755043029785),
 ('맛', 0.9154644012451172),
 ('스테이크', 0.914262056350708),
 ('꼬치', 0.913129448890686),
 ('단골집', 0.9109611511230469),
 ('고기', 0.9105876088142395),
 ('탕수육', 0.9091767072677612),
 ('맛그램', 0.9084287285804749),
 ('필라프', 0.9078981876373291),
 ('볶음밥', 0.9076505899429321),
 ('음식사진', 0.9071639776229858),
 ('초밥', 0.9068845510482788),
 ('매운맛', 0.9052410125732422),
 ('양고기', 0.9048858880996704),
 ('소주', 0.9045168161392212),
 ('짜장면', 0.9043819308280945),
 ('조마루감자탕', 0.9043161869049072)]

# Word2Vec.wv.syn0 단어가 저장되어 있는 행렬

In [11]:

word2vec_model.wv.syn0.shape

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  from ipykernel import kernelapp as app


(135446, 100)

In [12]:
# wrod2vec.wv.syn0norm - cosine similarity를 위하여 row normalize를 한 행렬로 모양은 같지만, 벡터의 2norm이 1이다..?
print(word2vec_model.wv.syn0norm.shape)
print(sum(v**2 for v in word2vec_model.wv.syn0[0]))
print(sum((v**2 for v in word2vec_model.wv.syn0norm[0])))

(135446, 100)
378.44826942863017
1.0000000274140424


C:\Users\mediance_ssh\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  from ipykernel import kernelapp as app
C:\Users\mediance_ssh\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  app.launch_new_instance()
C:\Users\mediance_ssh\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).


In [13]:
# word2vec.wv.index2word는 단어별 index가 저장되어 있음 (빈도순 정렬)
print(type(word2vec_model.wv.index2word))
print(len(word2vec_model.wv.index2word))
print(word2vec_model.wv.index2word[0])

<class 'list'>
135446
일상


In [14]:
# 단어 벡터를 가져오려면 .. size=100 이므로 100차원의 vector로 이루어짐 ㅎㄷㄷ
word2vec_model.wv['일상']

array([-0.5368336 ,  1.857119  , -0.15346059, -1.6179028 , -1.3754413 ,
        0.12573646, -0.03007481,  3.2329378 , -0.26311785,  1.0306453 ,
       -1.4477588 , -0.08926016,  1.7882625 ,  1.2157729 ,  0.61828995,
        2.51335   ,  1.1791475 ,  0.9172201 ,  4.558954  , -0.6255308 ,
        1.4600515 ,  1.2119313 , -0.12240368, -1.7979574 ,  0.72797227,
        0.55405426,  1.0818824 ,  4.2108545 ,  2.9012825 , -0.64767563,
       -1.5000602 ,  1.8767678 , -1.2540319 , -0.90317345,  1.5360197 ,
       -0.24718794, -1.0842106 ,  1.4807311 , -0.68278146, -0.13287747,
        0.0246343 , -2.705106  ,  2.0748348 ,  2.2202437 ,  1.029211  ,
        1.7129372 ,  3.3151522 ,  1.0011592 ,  0.96472985,  1.0497502 ,
        2.173751  ,  0.7742974 , -0.8995111 , -0.553434  , -1.0137199 ,
        4.116677  , -0.7501671 , -0.49500984,  0.3753781 ,  0.6437608 ,
       -2.1111057 , -0.7104022 ,  2.4959826 ,  2.9436593 , -4.9653845 ,
        4.114139  , -3.7980208 ,  3.2643046 ,  3.830705  ,  1.76

In [15]:
# word2vec.wv.vocab - {단어:vocab} 형식의 dict 
print(word2vec_model.wv.vocab['일상'])
print(word2vec_model.wv.vocab['일상'].count)

Vocab(count:42906, index:0, sample_int:1869852578)
42906


In [16]:
# preparing tag index to tag name
# word2vec의 경우 word2vec_model.wv.index2word 에 tagname이 들어있다..

In [17]:
# t-sne

In [18]:
from sklearn.manifold import TSNE

In [19]:
%%time
tsne = TSNE(n_components=2)
y_tsne = tsne.fit_transform(word2vec_model.wv.syn0)

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  from ipykernel import kernelapp as app


KeyboardInterrupt: 

In [ ]:
print(y_tsne.shape)
print(y_tsne[0])

In [ ]:
#

In [ ]:
# preparing 'tag index to tag' 이거...
index2tag = word2vec_model.wv.index2word
print(index2tag[:5])

In [ ]:
tag2index = {tag:index for index, tag in enumerate(index2tag)}
print(tag2index['일상'])
print(tag2index.get('맞친',-1))

In [ ]:
# prepaing bokeh

In [ ]:
from bokeh.plotting import figure, show, output_notebook

def mscatter(p, x, y, marker="circle", size=5, line_color="navy", fill_color="orange", alpha=0.5):
    p.scatter(x,y,marker=marker, size=size, line_color=line_color, fill_color=fill_color, alpha=alpha)
    
def mtext(p,x,y,text):
    p.text(x,y,text=[text], text_color="black", text_align="center", text_font_size="10pt")

output_notebook()    

In [ ]:
# scatter plot

In [ ]:
def scatterplot_hashtag(tag_idx, embedding):
    query_tag = word2vec_model.wv.index2word[tag_idx]
    print('{} ({})'.format(query_tag, tag_idx))
    
    # set figure
    p = figure(title='Similar tags of {}'.format(query_tag))
    p.grid.grid_line_color = None
    p.background_fill_color = "white"
    p.width = 900
    p.height = 900
    
    # scatter plot
    x = embedding[:,0]
    y = embedding[:,1]
    mscatter(p,x,y)
    
    
    # get similar tags
    similars = [query_tag]
    similars += [tag for tag, _ in word2vec_model.wv.most_similar(query_tag)]
    print(similars)
    
    # annotation
    for similar_tag in similars:
        idx = tag2index.get(similar_tag, -1)
        if idx == -1:
            continue
        x, y = embedding[idx,0], embedding[idx,1]
        mtext(p,x,y,similar_tag)
    
    # show
    show(p)

In [ ]:
scatterplot_hashtag(1, y_tsne)

In [ ]:
# MDS

In [ ]:
%%time
from sklearn.manifold import MDS
mds = MDS(n_components=2)
y_mds = mds.fit_transform(word2vec_model.wv.syn0)
y_mds.shape

In [ ]:
scatterplot_hashtag(1, y_mds)

In [ ]:
# lle

In [ ]:
%%time
from sklearn.manifold import LocallyLinearEmbedding
lle = LocallyLinearEmbedding(n_components=2)
y_lle = lle.fit_transform(word2vec_model.wv.syn0)
y_lle.shape

In [ ]:
scatterplot_hashtag(1, y_lle)

In [ ]:
# isomap

In [ ]:
%%time
from sklearn.manifold import Isomap
isomap = Isomap(n_components=2)
y_isomap = isomap.fit_transform(word2vec_model.wv.syn0)
y_isomap.shape

In [ ]:
scatterplot_hashtag(1, y_isomap)

In [ ]:
# 여기까지 결과..
'''
word2vec 으로 임베딩된 태그뭉치는 연관단어 찾기까지는 실효성이 있지만 
이걸 전부 차원축소해서 그래프 그리는것은 현실성 없어 보인다. 
전체는 2300만건인데 몇만건의 데이터만 가져와도 몇십~몇시간이 걸리며 사실상 알아볼 수가 없다.
이건 추후 텐서플로우-gpu를 사용한 방식으로 테스트해보는것도 필요할것같다.

예제에서는 wor2vec->doc2vec 을 하고 doc2vec 임베딩데이터로 비슷한 영화를 찾는 경우..
이걸 tag에 적용하려면 마찬가지로 tag를 category화 시켜서 상위 category를 보여줘야하는데
테스트 해볼만하긴한데 이게 무슨소용?

여기에선 bockh 라이브러리에서 color값 줘서 뭉치확인하고 parameter 튜닝 하는거랑
나중에 이쪽에서도 최적의 상태를 도출할 수 있는지 알아보는것.
'''